In [1]:
cd /maven_iuvs/stage/products/level1b/

/maven_iuvs/stage/products/level1b


In [2]:
%autocall 1

Automatic calling is: Smart


In [3]:
mode040_fnames = !ls *IPH2-*-mode040-muv*.fits.gz
mode040_fnames[:10]

['mvn_iuv_l1b_IPH2-cycle00001-mode040-muv_20140419T004426_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00002-mode040-muv_20140419T084429_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00003-mode040-muv_20140419T164430_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00004-mode040-muv_20140420T004433_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00005-mode040-muv_20140420T084434_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00006-mode040-muv_20140420T164436_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00007-mode040-muv_20140421T004438_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00008-mode040-muv_20140421T084441_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00009-mode040-muv_20140421T164443_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00010-mode040-muv_20140422T004445_v00_r00.fits.gz']

In [13]:
from IPython.parallel import Client
c = Client()

In [14]:
all_engines=c[:]
lb = c.load_balanced_view()

In [15]:
%%px
import os
from iuvs import io
import numpy as np

In [16]:
import os
from iuvs import io

from skimage.exposure import rescale_intensity

def imshow_and_cbar(data, ax, txt, vmin=None, vmax=None):
    im = ax.imshow(data, vmin=vmin, vmax=vmax)
    cbar = plt.colorbar(im, ax=ax)
    cbar.ax.tick_params(labelsize=9)
    ax.set_title(txt)
    
def generate_dark_overview(fname, interactive=False):
    from iuvs import io
    fig, axes = plt.subplots(2,2)
    iuvsfname = io.Filename(fname)
    axes = axes.flatten()
    data = io.L1BReader(str(io.level1bpath / fname))
    subbed_img = data.detector_background_subtracted
    p2, p98 = np.percentile(subbed_img, (2, 98))
    imshow_and_cbar(subbed_img, axes[0], 'subtracted, 2/98-stretch',
                   vmin=p2, vmax=p98)
    axes[1].hist(subbed_img.flatten(), bins=100, log=True)
    p2, p98 = np.percentile(data.detector_dark, (2, 98))
    imshow_and_cbar(data.detector_dark, axes[2], 'dark, 2/98-stretch',
                    vmin=p2, vmax=p98)
    axes[3].hist(data.detector_dark.flatten(), bins=100, log=True)
    fig.suptitle('Exposure: {}, Time:{}'.format(data.img_header['INT_TIME'],
                                              iuvsfname.time.isoformat()))
    plt.savefig(os.environ['HOME']+'/public_html/level1b_darks/plots/l1b_img_vs_darks_{}'.format(iuvsfname.time.isoformat()),
            dpi=150)
    if not interactive:
        plt.close(fig)
        return iuvsfname.basename
    else:
        return data

In [8]:
%matplotlib qt

In [9]:
data = generate_dark_overview(mode040_fnames[0], interactive=True)

In [17]:
all_engines.push(dict(
        imshow_and_cbar=imshow_and_cbar,
        generate_dark_overview=generate_dark_overview,
))

<AsyncResult: finished>

In [18]:
todo = mode040_fnames[:8]
results = lb.map_async(generate_dark_overview, todo)

In [19]:
import time
while not results.ready():
    print(round(100*results.progress/len(todo)), ' %')
    time.sleep(10)

0  %


In [27]:
results.get()

['mvn_iuv_l1b_IPH2-cycle00001-mode040-muv_20140419T004426_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00002-mode040-muv_20140419T084429_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00003-mode040-muv_20140419T164430_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00004-mode040-muv_20140420T004433_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00005-mode040-muv_20140420T084434_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00006-mode040-muv_20140420T164436_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00007-mode040-muv_20140421T004438_v00_r00.fits.gz',
 'mvn_iuv_l1b_IPH2-cycle00008-mode040-muv_20140421T084441_v00_r00.fits.gz']